# Code complet de traitement des données

## Lecture du fichier json

In [1]:
import json
import pandas as pd
from typing import List, Dict, Any
import os
import requests
import cv2
import numpy as np
import matplotlib.pyplot as plt
from transformers import pipeline
import logging
import torch
from PIL import Image

# Configurer le logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

/Users/tepe/Desktop/Code/Project/Start-up/Product_Analysis/deep_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/tepe/Desktop/Code/Project/Start-up/Product_Analysis/deep_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
iteration = 6

In [3]:
nom_fichier = f'Data/{iteration}/smartphone_infos_{iteration}'

In [4]:
def json_to_dataframe(chemin_fichier: str) -> pd.DataFrame:
    """
    Lit un fichier JSON et le transforme en un DataFrame Pandas.
    Chaque donnée principale, critère et image est associé à une colonne distincte.

    Parameters:
    - chemin_fichier (str): Chemin vers le fichier JSON.

    Returns:
    - pd.DataFrame: DataFrame contenant les données du JSON.
    """
    # Charger le fichier JSON
    try:
        with open(chemin_fichier, 'r', encoding='utf-8') as fichier:
            data = json.load(fichier)
    except FileNotFoundError:
        print(f"Le fichier {chemin_fichier} n'a pas été trouvé.")
        return pd.DataFrame()
    except json.JSONDecodeError as e:
        print(f"Erreur lors du décodage JSON : {e}")
        return pd.DataFrame()
    
    # Liste pour stocker les données transformées
    liste_donnees: List[Dict[str, Any]] = []
    
    # Ensemble pour collecter tous les types de critères uniques
    types_criteres = set()
    
    for annonce in data:
        # Dictionnaire pour une seule ligne du DataFrame
        ligne: Dict[str, Any] = {}
        
        # Extraire les données principales
        ligne['title'] = annonce.get('title', None)
        ligne['link'] = annonce.get('link', None)
        ligne['price'] = annonce.get('price', None)
        ligne['description'] = annonce.get('description', None)
        ligne['publication_date'] = annonce.get('publication_date', None)
        
        # Extraire les critères et les ajouter comme colonnes distinctes
        criteria = annonce.get('criteria', [])
        for critere in criteria:
            type_critere = critere.get('type', '').replace('criteria_item_', '')
            valeur_critere = critere.get('value', None)
            # Remplacer les caractères indésirables dans le nom de la colonne -- Par exemple, 'phone_brand' devient 'Brand'
            colonne_critere = type_critere.replace('phone_', '').replace('_', ' ').title().replace(' ', '_')
            # Ajouter le type de critère à l'ensemble
            types_criteres.add(colonne_critere)
            # Assigner la valeur au critère correspondant
            ligne[colonne_critere] = valeur_critere
        
        # Extraire les images et les stocker dans une colonne sous forme de liste
        images = annonce.get('images', [])
        ligne['images'] = images
        
        # Ajouter la ligne à la liste
        liste_donnees.append(ligne)
    
    # Créer le DataFrame
    df = pd.DataFrame(liste_donnees)
    
    # Convertir 'publication_date' en datetime
    df['publication_date'] = pd.to_datetime(df['publication_date'], format='%d/%m/%Y à %H:%M', errors='coerce')
    
    # Afficher les types de critères uniques trouvés
    print("Types de critères uniques trouvés :", types_criteres)
    
    return df

In [5]:
df = json_to_dataframe(f'{nom_fichier}.json')
df

Types de critères uniques trouvés : {'Refurbished_Item_Condition', 'Brand', 'Color', 'Reparability_Index', 'Memory', 'Spare_Parts_Availability', 'Model', 'Condition'}


,title,link,price,description,publication_date,Condition,Brand,Model,Color,Memory,images,Spare_Parts_Availability,Refurbished_Item_Condition,Reparability_Index
0,Téléphone filaire,https://www.leboncoin.fr/ad/telephones_objets_...,8 €,Vends téléphone filaire vintage.\nPas d envoi ...,2024-11-24 14:48:00,Bon état,Alcatel,Autre,Marron,8 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,NaN,NaN,NaN
1,IPhone 11,https://www.leboncoin.fr/ad/telephones_objets_...,255 €,iPhone 11 jaune 64go en très bonne état 100% d...,2024-11-16 21:21:00,Très bon état,Apple,iPhone 11,Jaune,64 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,NaN,NaN,NaN
2,None,https://www.leboncoin.fr/ad/telephones_objets_...,None,None,NaT,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN
3,Téléphone Xiaomi 12T,https://www.leboncoin.fr/ad/telephones_objets_...,200 €,Je vends mon Xiaomi 12T en PARFAIT état. \nÉcr...,2024-11-17 18:42:00,État neuf,Xiaomi,Autre,Gris,256 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,NaN,NaN,NaN
4,IPhone 7 💫,https://www.leboncoin.fr/ad/telephones_objets_...,50 €,"Je vends un IPHONE\nJ’ai la GARANTIE, et la FA...",2024-10-08 01:06:00,Très bon état,Apple,iPhone 7,Noir,32 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,Non renseignée,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,Iphone 11,https://www.leboncoin.fr/ad/telephones_objets_...,300 €,"Bonjour, je vend mon iphone 11 pour m’acheter ...",2024-11-25 18:43:00,Très bon état,Apple,iPhone 11,Noir,64 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,NaN,NaN,NaN
2996,None,https://www.leboncoin.fr/ad/telephones_objets_...,None,None,NaT,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN
2997,Protection xiaomi redmi note 13 pro plus,https://www.leboncoin.fr/ad/telephones_objets_...,5 €,Vds protection pour téléphone XIAOMI Redmi Not...,2024-11-25 18:42:00,Très bon état,Xiaomi,Autre,Noir,NaN,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,NaN,NaN,NaN
2998,None,https://www.leboncoin.fr/ad/telephones_objets_...,None,None,NaT,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN


## Filtrage des données

In [6]:
# Visualiser les valeurs uniques de la colonne 'Model'
df['Condition'].unique()

array(['Bon état', 'Très bon état', nan, 'État neuf', 'Pour pièces',
       'Reconditionné', 'État satisfaisant'], dtype=object)

### Filtrage des models et marques

In [7]:
df_filtered = df.copy()

In [8]:
df_filtered = df_filtered[df_filtered['Model'] != 'Autre']
df_filtered = df_filtered[df_filtered['Brand'] != 'Autre']
df_filtered = df_filtered[df_filtered['Condition'].notna()]
df_filtered['Condition'] = df_filtered['Condition'].replace({'Pour pièces': 0, 'État satisfaisant': 1, 'Bon état': 2, 'Très bon état':3 ,'État neuf': 4, 'Reconditionné': 4})
#df_filtered = df_filtered[df_filtered['Model'].notna()] # Certains modèles sont vides mais les images sont corrects

/var/folders/d4/tf_qqhg566j2y0pml6db74z40000gn/T/ipykernel_18317/2174254153.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_filtered['Condition'] = df_filtered['Condition'].replace({'Pour pièces': 0, 'État satisfaisant': 1, 'Bon état': 2, 'Très bon état':3 ,'État neuf': 4, 'Reconditionné': 4})


In [9]:
df_filtered

,title,link,price,description,publication_date,Condition,Brand,Model,Color,Memory,images,Spare_Parts_Availability,Refurbished_Item_Condition,Reparability_Index
1,IPhone 11,https://www.leboncoin.fr/ad/telephones_objets_...,255 €,iPhone 11 jaune 64go en très bonne état 100% d...,2024-11-16 21:21:00,3,Apple,iPhone 11,Jaune,64 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,NaN,NaN,NaN
4,IPhone 7 💫,https://www.leboncoin.fr/ad/telephones_objets_...,50 €,"Je vends un IPHONE\nJ’ai la GARANTIE, et la FA...",2024-10-08 01:06:00,3,Apple,iPhone 7,Noir,32 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,Non renseignée,NaN,NaN
9,Iphone 15 pro Max,https://www.leboncoin.fr/ad/telephones_objets_...,1 000 €,"Vends iPhone 15 Pro Max couleur titane, très b...",2024-11-16 21:21:00,3,Apple,iPhone 15 Pro Max,Argent / Silver,256 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,NaN,NaN,NaN
10,IPhone apple,https://www.leboncoin.fr/ad/telephones_objets_...,70 €,"iPhone en excellent état,vendu avec garanti et...",2024-11-23 16:42:00,3,Apple,iPhone SE,Rouge,64 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,NaN,NaN,NaN
11,Réveil,https://www.leboncoin.fr/ad/telephones_objets_...,10 €,Vends réveil en très bon état.,2024-11-16 21:22:00,3,NaN,NaN,Noir,8 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2984,Iphone 13 à vendre sur Paris,https://www.leboncoin.fr/ad/telephones_objets_...,350 €,"Vendu avec facture, accessoires d’origine et g...",2024-11-25 18:39:00,4,Apple,iPhone 13,Violet,128 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,Non disponible,Très bon état,NaN
2985,IPhone 13 128GO blanc,https://www.leboncoin.fr/ad/telephones_objets_...,450 €,"Vend iPhone 13 128GO , en excellent état \nT...",2024-11-25 18:39:00,4,Apple,iPhone 13,Blanc,128 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,NaN,NaN,NaN
2993,Iphone 13 128Go Blanc,https://www.leboncoin.fr/ad/telephones_objets_...,390 €,Je vends mon iPhone 13 128Go car j'ai acheté u...,2024-11-25 18:41:00,3,Apple,iPhone 13,Blanc,128 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,NaN,NaN,NaN
2995,Iphone 11,https://www.leboncoin.fr/ad/telephones_objets_...,300 €,"Bonjour, je vend mon iphone 11 pour m’acheter ...",2024-11-25 18:43:00,3,Apple,iPhone 11,Noir,64 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,NaN,NaN,NaN


## Téléchargement des images dans le dossier

In [10]:
def telecharger_image(url):
    """
    Télécharge une image depuis une URL et retourne l'image sous forme de tableau NumPy.

    Parameters:
    - url (str): L'URL de l'image à télécharger.

    Returns:
    - image (np.ndarray ou None): Image téléchargée ou None en cas d'échec.
    """
    try:
        # Télécharger le contenu de l'image
        reponse = requests.get(url, stream=True)
        reponse.raise_for_status()  # Lève une exception pour les codes d'erreur HTTP

        # Convertir le contenu en tableau NumPy
        image_array = np.asarray(bytearray(reponse.content), dtype=np.uint8)
        image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

        if image is not None:
            print("Image téléchargée avec succès.")
            return image
        else:
            print("Erreur : L'image n'a pas pu être décodée.")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Une erreur est survenue lors du téléchargement : {e}")
        return None


In [11]:
def enregistrer_image(image, nom_image, dossier_destination):
    """
    Enregistre une image dans le dossier spécifié. Crée le dossier s'il n'existe pas.

    Parameters:
    - image (np.ndarray): Image à enregistrer (format BGR comme utilisé par OpenCV).
    - nom_image (str): Nom de fichier pour enregistrer l'image (ex. 'image.jpg').
    - dossier_destination (str): Chemin du dossier où enregistrer l'image.

    Returns:
    - chemin_complet (str ou None): Chemin complet de l'image enregistrée ou None en cas d'échec.
    """
    try:
        # Créer le dossier s'il n'existe pas
        os.makedirs(dossier_destination, exist_ok=True)

        # Chemin complet de sauvegarde
        chemin_complet = os.path.join(dossier_destination, nom_image)

        # Enregistrer l'image avec OpenCV
        success = cv2.imwrite(chemin_complet, image)

        if success:
            print(f"Image enregistrée à : {chemin_complet}")
            return nom_image
        else:
            print(f"Erreur : L'image n'a pas pu être sauvegardée à : {chemin_complet}")
            return None

    except Exception as e:
        print(f"Une erreur est survenue lors de la sauvegarde de l'image : {e}")
        return None


In [12]:
df_annonces = df_filtered.copy()

In [13]:
# Exemple d'utilisation
number_annonce = 0
all_chemins = []

for annonce in df_annonces['images']:
    numero = 0
    chemins_annonce = []
    for url_image in annonce:
        dossier_destination = f'Data/{iteration}/images_telechargees'
        nom_image = f'produit_{number_annonce}_image_{numero}.jpg'
        
        if not os.path.isfile(f'{dossier_destination}/{nom_image}'):
            image = telecharger_image(url_image)
            # Vérification si l'image n'est pas None et contient des données
            if image is not None and image.size > 1:
                enregistrer_image(image, nom_image, dossier_destination)
                chemins_annonce.append(nom_image)
                numero += 1
        else:
            chemins_annonce.append(nom_image)
            numero += 1
    
    number_annonce += 1
            
    all_chemins.append(chemins_annonce)

In [14]:
df_annonces['Images_Path'] = all_chemins
df_annonces

,title,link,price,description,publication_date,Condition,Brand,Model,Color,Memory,images,Spare_Parts_Availability,Refurbished_Item_Condition,Reparability_Index,Images_Path
1,IPhone 11,https://www.leboncoin.fr/ad/telephones_objets_...,255 €,iPhone 11 jaune 64go en très bonne état 100% d...,2024-11-16 21:21:00,3,Apple,iPhone 11,Jaune,64 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,NaN,NaN,NaN,[produit_0_image_0.jpg]
4,IPhone 7 💫,https://www.leboncoin.fr/ad/telephones_objets_...,50 €,"Je vends un IPHONE\nJ’ai la GARANTIE, et la FA...",2024-10-08 01:06:00,3,Apple,iPhone 7,Noir,32 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,Non renseignée,NaN,NaN,[produit_1_image_0.jpg]
9,Iphone 15 pro Max,https://www.leboncoin.fr/ad/telephones_objets_...,1 000 €,"Vends iPhone 15 Pro Max couleur titane, très b...",2024-11-16 21:21:00,3,Apple,iPhone 15 Pro Max,Argent / Silver,256 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,NaN,NaN,NaN,[produit_2_image_0.jpg]
10,IPhone apple,https://www.leboncoin.fr/ad/telephones_objets_...,70 €,"iPhone en excellent état,vendu avec garanti et...",2024-11-23 16:42:00,3,Apple,iPhone SE,Rouge,64 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,NaN,NaN,NaN,"[produit_3_image_0.jpg, produit_3_image_1.jpg]"
11,Réveil,https://www.leboncoin.fr/ad/telephones_objets_...,10 €,Vends réveil en très bon état.,2024-11-16 21:22:00,3,NaN,NaN,Noir,8 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,NaN,NaN,NaN,"[produit_4_image_0.jpg, produit_4_image_1.jpg,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2984,Iphone 13 à vendre sur Paris,https://www.leboncoin.fr/ad/telephones_objets_...,350 €,"Vendu avec facture, accessoires d’origine et g...",2024-11-25 18:39:00,4,Apple,iPhone 13,Violet,128 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,Non disponible,Très bon état,NaN,"[produit_1714_image_0.jpg, produit_1714_image_..."
2985,IPhone 13 128GO blanc,https://www.leboncoin.fr/ad/telephones_objets_...,450 €,"Vend iPhone 13 128GO , en excellent état \nT...",2024-11-25 18:39:00,4,Apple,iPhone 13,Blanc,128 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,NaN,NaN,NaN,"[produit_1715_image_0.jpg, produit_1715_image_..."
2993,Iphone 13 128Go Blanc,https://www.leboncoin.fr/ad/telephones_objets_...,390 €,Je vends mon iPhone 13 128Go car j'ai acheté u...,2024-11-25 18:41:00,3,Apple,iPhone 13,Blanc,128 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,NaN,NaN,NaN,"[produit_1716_image_0.jpg, produit_1716_image_..."
2995,Iphone 11,https://www.leboncoin.fr/ad/telephones_objets_...,300 €,"Bonjour, je vend mon iphone 11 pour m’acheter ...",2024-11-25 18:43:00,3,Apple,iPhone 11,Noir,64 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,NaN,NaN,NaN,"[produit_1717_image_0.jpg, produit_1717_image_..."


## Découpage du fond des images

In [15]:
def afficher_image(image, legende: str = ''):
    # Afficher l'image avec matplotlib
    plt.imshow(image)
    plt.axis('on')  # Masquer les axes
    plt.title(legende)
    plt.show()

In [16]:
# Note : Remplacez "facebook/detr-resnet-50-panoptic" par le modèle approprié si nécessaire
pipe = pipeline("image-segmentation", model="briaai/RMBG-1.4", trust_remote_code=True)

def remove_background(image_path):
    # 1. Charger l'image
    image = Image.open(image_path).convert("RGBA")  # Utiliser RGBA pour gérer la transparence

    # 2. Appliquer le pipeline pour obtenir l'image avec le fond supprimé
    result = pipe(image)

    # 3. Vérifier si le résultat a un canal alpha
    if result.mode == 'RGBA':
        # Créer une image de fond blanche
        background = Image.new("RGB", result.size, (255, 255, 255))
        # Combiner l'image résultante avec le fond blanc en utilisant le canal alpha comme masque
        background.paste(result, mask=result.split()[3])  # Le canal alpha est à l'index 3
        # Convertir l'image combinée en BGR pour OpenCV
        image_array = np.array(background)
        bgr_image = cv2.cvtColor(image_array, cv2.COLOR_RGB2BGR)
        # Extraire le masque à partir du canal alpha
        alpha = result.split()[-1]
        mask = np.array(alpha)
    else:
        # Si pas de canal alpha, créer un masque plein (pas de suppression de fond)
        mask = np.ones((result.height, result.width), dtype=np.uint8) * 255
        # Convertir l'image en RGB et ensuite en BGR pour OpenCV
        rgb_image = result.convert("RGB")
        image_array = np.array(rgb_image)
        bgr_image = cv2.cvtColor(image_array, cv2.COLOR_RGB2BGR)

    return bgr_image, mask


Device set to use mps:0


In [17]:
numero = 0
for annonce in df_annonces['Images_Path']:
    for image_name in annonce:
        dossier_destination = f'Data/{iteration}/images_without_background'
        path = f'Data/{iteration}/images_telechargees/{image_name}'
        
        # Vérification que l'image source existe
        if not os.path.isfile(path):
            print(f"L'image source {image_name} n'existe pas dans le dossier.")
            continue
            
        # Vérification que l'image n'a pas déjà été traitée
        if not os.path.isfile(f'{dossier_destination}/{image_name}'):
            try:
                image, mask = remove_background(path)
                enregistrer_image(image, image_name, dossier_destination)
                print(f"Traitement réussi pour {image_name}")
            except Exception as e:
                print(f"Erreur lors du traitement de {image_name}: {str(e)}")


In [18]:
df_annonces.to_csv(f'{nom_fichier}.csv', encoding='utf-8', errors='replace')
df_annonces

,title,link,price,description,publication_date,Condition,Brand,Model,Color,Memory,images,Spare_Parts_Availability,Refurbished_Item_Condition,Reparability_Index,Images_Path
1,IPhone 11,https://www.leboncoin.fr/ad/telephones_objets_...,255 €,iPhone 11 jaune 64go en très bonne état 100% d...,2024-11-16 21:21:00,3,Apple,iPhone 11,Jaune,64 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,NaN,NaN,NaN,[produit_0_image_0.jpg]
4,IPhone 7 💫,https://www.leboncoin.fr/ad/telephones_objets_...,50 €,"Je vends un IPHONE\nJ’ai la GARANTIE, et la FA...",2024-10-08 01:06:00,3,Apple,iPhone 7,Noir,32 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,Non renseignée,NaN,NaN,[produit_1_image_0.jpg]
9,Iphone 15 pro Max,https://www.leboncoin.fr/ad/telephones_objets_...,1 000 €,"Vends iPhone 15 Pro Max couleur titane, très b...",2024-11-16 21:21:00,3,Apple,iPhone 15 Pro Max,Argent / Silver,256 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,NaN,NaN,NaN,[produit_2_image_0.jpg]
10,IPhone apple,https://www.leboncoin.fr/ad/telephones_objets_...,70 €,"iPhone en excellent état,vendu avec garanti et...",2024-11-23 16:42:00,3,Apple,iPhone SE,Rouge,64 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,NaN,NaN,NaN,"[produit_3_image_0.jpg, produit_3_image_1.jpg]"
11,Réveil,https://www.leboncoin.fr/ad/telephones_objets_...,10 €,Vends réveil en très bon état.,2024-11-16 21:22:00,3,NaN,NaN,Noir,8 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,NaN,NaN,NaN,"[produit_4_image_0.jpg, produit_4_image_1.jpg,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2984,Iphone 13 à vendre sur Paris,https://www.leboncoin.fr/ad/telephones_objets_...,350 €,"Vendu avec facture, accessoires d’origine et g...",2024-11-25 18:39:00,4,Apple,iPhone 13,Violet,128 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,Non disponible,Très bon état,NaN,"[produit_1714_image_0.jpg, produit_1714_image_..."
2985,IPhone 13 128GO blanc,https://www.leboncoin.fr/ad/telephones_objets_...,450 €,"Vend iPhone 13 128GO , en excellent état \nT...",2024-11-25 18:39:00,4,Apple,iPhone 13,Blanc,128 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,NaN,NaN,NaN,"[produit_1715_image_0.jpg, produit_1715_image_..."
2993,Iphone 13 128Go Blanc,https://www.leboncoin.fr/ad/telephones_objets_...,390 €,Je vends mon iPhone 13 128Go car j'ai acheté u...,2024-11-25 18:41:00,3,Apple,iPhone 13,Blanc,128 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,NaN,NaN,NaN,"[produit_1716_image_0.jpg, produit_1716_image_..."
2995,Iphone 11,https://www.leboncoin.fr/ad/telephones_objets_...,300 €,"Bonjour, je vend mon iphone 11 pour m’acheter ...",2024-11-25 18:43:00,3,Apple,iPhone 11,Noir,64 Go,[https://img.leboncoin.fr/api/v1/lbcpb1/images...,NaN,NaN,NaN,"[produit_1717_image_0.jpg, produit_1717_image_..."
